In [39]:
import os
from os import kill
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException
# from selenium.common.exceptions import StaleElementReferenceException
# from selenium.common.exceptions import ElementNotInteractableException
# from selenium.common.exceptions import ElementClickInterceptedException
import time
import pandas as pd
import numpy as np


def article_search(key, outfile):
    driver = webdriver.Chrome("C:/chromedriver")  #브라우저 켜기
    driver.get('https://pubmed.ncbi.nlm.nih.gov/') 
    toAdvance = driver.find_element_by_css_selector('a.search-input-link')
    toAdvance.click()

    # title과 abstract으로 설정
    title_abstrac = driver.find_element_by_css_selector('select#field-selector')
    title_abstrac.click()
    opt = driver.find_element_by_css_selector('#field-selector > option:nth-child(39)')
    opt.click()

    # Input Keywords
    search = driver.find_element_by_css_selector('input#id_term')
    search.click()
    search.send_keys(key)
    search.send_keys(Keys.ENTER)

    searchBtn = driver.find_element_by_css_selector('#search-form > div > div > div.query-box-section-wrapper > div.button-wrapper > button')
    searchBtn.click()

    title_arr = []
    year_arr = []
    author_arr = []
    journal_arr = []
    link_arr = []
    abstract_arr = []

    titles = driver.find_elements_by_css_selector('a.docsum-title')
    cnt = 0
    result = int(driver.find_element_by_css_selector('div.results-amount > span.value').text)
    for _ in range(result):
        titles = driver.find_elements_by_css_selector('a.docsum-title')
        num = driver.find_element_by_css_selector('#search-results > section > div.search-results-chunks > div > article:nth-child(2) > div.item-selector-wrap.selectors-and-actions.first-selector > label > span').text
        
        # print("lenght: ", len(titles))
        # print("num:", num)
        if len(titles) > 10:
            titles2 = driver.find_elements_by_css_selector('a.docsum-title')
            titles2[10].click()
            driver.back()
        elif len(titles) <= 10:
            for i in range(len(titles)):
                time.sleep(1)
                titles2 = driver.find_elements_by_css_selector('a.docsum-title')
                titles2[i].click()

                #title
                title = driver.find_element_by_css_selector('h1.heading-title')
                title_arr.append(title.text)

                # year
                try:
                    year = driver.find_element_by_css_selector('#full-view-heading > div.article-citation > div.article-source > span.cit')
                    year_arr.append(year.text)
                except:
                    year = ""
                    year_arr.append(year)

                #authors
                try:
                    authors = driver.find_element_by_css_selector('#full-view-heading > div.inline-authors > div > div > span:nth-child(1) > a')
                    author_arr.append(authors.text)
                except:
                    author = ""
                    author_arr.append(author)
                # journals
                try:
                    journal = driver.find_element_by_css_selector('#full-view-heading > div.article-citation > div > div > button') 
                    journal_arr.append(journal.text)
                except:
                    journal = ""
                    journal_arr.append(journal)
                # abstract
                try:
                    abstract = driver.find_element_by_css_selector('div.abstract-content')
                    abstract_arr.append(abstract.text)
                except:
                    abstract = ""
                    abstract_arr.append(abstract)

                #link
                try:
                    link = driver.find_elements_by_css_selector('a.id-link')
                    href= link[-1].get_attribute('href')
                
                except IndexError:
                    href = None
                link_arr.append(href)

                driver.back()
                cnt += 1
                
                if i == 9:
                    ## Show More
                    more = driver.find_element_by_css_selector('#search-results > section > div.search-results-paginator.next-results-paginator.has-nav > button > span')
                    more.click()
                    time.sleep(1.2)
            print(f"Article Count: {cnt}, Proceeding rate: {round(cnt/int(result) * 100, 2)}%")
        if len(title_arr) == result:
            df = pd.DataFrame(np.c_[title_arr, year_arr, journal_arr, author_arr, link_arr, abstract_arr], columns=['title','year', 'journal','authors','links','abstract'])
            df.to_csv(outfile, encoding='utf8')
            print('Done!')
            break

In [40]:
article_search(key="(incontinence associated dermatitis[Title/Abstract]) OR diaper dermatitis[Title/Abstract]",
outfile='IAD2.csv')

Article Count: 10, Proceeding rate: 1.63%
Article Count: 20, Proceeding rate: 3.26%
Article Count: 30, Proceeding rate: 4.89%
Article Count: 40, Proceeding rate: 6.53%
Article Count: 50, Proceeding rate: 8.16%
Article Count: 60, Proceeding rate: 9.79%
Article Count: 70, Proceeding rate: 11.42%
Article Count: 80, Proceeding rate: 13.05%
Article Count: 90, Proceeding rate: 14.68%
Article Count: 100, Proceeding rate: 16.31%
Article Count: 110, Proceeding rate: 17.94%
Article Count: 120, Proceeding rate: 19.58%
Article Count: 130, Proceeding rate: 21.21%
Article Count: 140, Proceeding rate: 22.84%
Article Count: 150, Proceeding rate: 24.47%
Article Count: 160, Proceeding rate: 26.1%
Article Count: 170, Proceeding rate: 27.73%
Article Count: 180, Proceeding rate: 29.36%
Article Count: 190, Proceeding rate: 31.0%
Article Count: 200, Proceeding rate: 32.63%
Article Count: 210, Proceeding rate: 34.26%
Article Count: 220, Proceeding rate: 35.89%
Article Count: 230, Proceeding rate: 37.52%
Artic

In [43]:
import pandas as pd

In [53]:
df=pd.read_csv('IAD2.csv')
df.shape

(613, 7)

In [63]:
import re
regex = re.compile(r'\d{4}')


TypeError: object of type 'NoneType' has no len()

In [72]:
## 연도찾기
import re
regex = re.compile(r'\d{4}')
result = []
for i in range(df.shape[0]):
  temp = str(df['year'][i])
  rst = regex.findall(temp)
  if len(rst)>=1:
    result.append(str(rst[0]))
  else:
    result.append("")
  